In [1]:
# !pip install pandas
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [117]:
from urllib.request import urlopen
import requests
import pandas as pd
from bs4 import BeautifulSoup
from hdfs import InsecureClient
from datetime import datetime,timedelta
import json
from pyspark.sql import Row

In [31]:
client = InsecureClient('http://localhost:9870', user='worker') #나의 로컬호스트주소 #user : big(내가정함)


In [5]:
def cal_std_day(befor_day):   
    x = datetime.now() - timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [15]:
# 
base_url = 'https://kin.naver.com/qna/expertAnswerList.naver?dirId=701&queryTime='
param = '%2014%3A51%3A20&page='
page_num = 1
url = base_url + cal_std_day(0) + param + str(page_num)
url

'https://kin.naver.com/qna/expertAnswerList.naver?dirId=701&queryTime=2022-10-16%2014%3A51%3A20&page=1'

In [21]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}
res = requests.get(url, headers = headers)
bs = BeautifulSoup(res.text, 'html.parser')


In [22]:
# 질문 리스트
trs = bs.find('tbody',{'id':'au_board_list'}).findAll('tr')

In [70]:
# 질문 URL
trs[0].find('td',{'class':'title'}).find('a')['href']
# 질문 제목 
trs[0].find('td',{'class':'title'}).find('a').text
# 분야
trs[0].find('td',{'class':'field'}).find('a').text
# # 답변의사 
trs[0].find('td',{'class':'questioner'}).find('a')['href'].split('=')[1]
# 의사이름
trs[0].find('td',{'class':'questioner'}).find('a').text
# 답변일자
trs[0].find('td',{'class':'t_num'}).text

'/qna/detail.naver?d1id=7&dirId=70116&docId=430403636'

'X-ray 좌폐 상부 병소'

'암센터'

'ZpLHwcZs6VN8eRB64%2FqBWJ45B1HSM8srkcTGsmusWwA%3D'

'김철수'

'2분 '

In [101]:
q_base_url = 'https://kin.naver.com/'
q_param = ['qna/detail.naver?d1id=7&dirId=70111&docId=430526243']


q_url = q_base_url + q_param[0]
q_url

res = requests.get(q_url, headers = headers)
bs = BeautifulSoup(res.text, 'html.parser')



'https://kin.naver.com/qna/detail.naver?d1id=7&dirId=70111&docId=430526243'

In [102]:
# 질문내용
bs.find('div',{'class':'c-heading__content'}).text.replace('\n','').replace('\t','')

temp_ls = []
ps = bs.find('div',{'class':'se-main-container'})
for p in ps.findAll('p'):
    p_content = p.find('span').text
    if p_content == ' ' :
        continue
    else :
        temp_ls.append(p_content.replace('\u200b',''))
answer= ' '.join(temp_ls)
answer

'치과에서 사랑니 뽑고 이렇게 약 처방 받았는데요!기존에 제로큐탄 하루에 2알씩 복용중이었구같이 먹어도 되나요? 의사 , 약사분들 답변 부탁드립니다! :) '

'안녕하세요. 하이닥-네이버 지식iN 상담의 이상준 입니다.  해당 항생제와는 상호작용 하지 않으니 드시던 약 계속 드셔도 괜찮습니다.'

In [108]:
base_url = ['https://kin.naver.com/qna/expertAnswerList.naver?dirId=701&queryTime=','https://kin.naver.com/']
param = '%2014%3A51%3A20&page='
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}


cols = ['qus_title','qus_dir','qus_doc_id','qus_doc','qus_content','qus_answer']
data = []
for i in range(1,100):
    
    url = base_url[0] + cal_std_day(0) + param + str(i)
    res = requests.get(url, headers = headers)
    bs = BeautifulSoup(res.text, 'html.parser')
    
    # 질문 리스트
    trs = bs.find('tbody',{'id':'au_board_list'}).findAll('tr')
    for tr in trs :
        rows =[] 
        if tr.find('td',{'class':'t_num'}).text[:3] != '202' :
            # 질문 URL
            qus_url = tr.find('td',{'class':'title'}).find('a')['href']
            # 질문 제목 
            rows.append(tr.find('td',{'class':'title'}).find('a').text)
            # 분야
            rows.append(tr.find('td',{'class':'field'}).find('a').text)
            # 답변의사 아이디
            rows.append(tr.find('td',{'class':'questioner'}).find('a')['href'].split('=')[1])
            # 의사이름
            rows.append(tr.find('td',{'class':'questioner'}).find('a').text)
            q_url = base_url[1] + qus_url
            q_res = requests.get(q_url, headers = headers)
            q_bs = BeautifulSoup(q_res.text, 'html.parser')
            print(q_url)
            # 질문내용
            try :
                rows.append(q_bs.find('div',{'class':'c-heading__content'}).text.replace('\n','').replace('\t',''))
            except :
                rows.append('No Content')
            temp_ls = []
            ps = q_bs.find('div',{'class':'se-main-container'})
            ps_1 = q_bs.find('div',{'class':'_endContentsText c-heading-answer__content-user'})
            try:
                for p in ps.findAll('p'):
                    p_content = p.find('span').text
                    if p_content == ' ' :
                        continue
                    else :
                        temp_ls.append(p_content.replace('\u200b','').replace('\xa0',''))
                rows.append(' '.join(temp_ls))
                tmp = dict(zip(cols,rows))
                data.append(tmp)
            except :
                for p in ps_1.findAll('p'):
                    p_content = p.text
                    if p_content == ' ' :
                        continue
                    else :
                        temp_ls.append(p_content.replace('\u200b','').replace('\xa0',''))
                rows.append(' '.join(temp_ls))
                tmp = dict(zip(cols,rows))
                data.append(tmp)

https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430615980
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430580798
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430610609
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430521616
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70111&docId=430526243
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430393191
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430343318
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430406954
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430450441
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430452192
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430409666
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430466097
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430465467
https://kin.naver.com//qn

https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010102&docId=430617772
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70103&docId=430617265
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010102&docId=430616664
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430592104
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70103&docId=430572914
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430556686
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010107&docId=430616562
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430529214
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70115&docId=430615577
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70115&docId=430615625
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010103&docId=430615692
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70115&docId=430605574
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010104&docId=430615206
https://kin.nav

https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010102&docId=430602685
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010102&docId=430606489
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010104&docId=430574416
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010104&docId=430576146
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430528557
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010104&docId=430577500
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010104&docId=430590539
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70122&docId=430605877
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010102&docId=430605236
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010104&docId=430590668
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70114&docId=430602031
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70103&docId=430605431
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010104&docId=430593782
https:/

https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010108&docId=430449633
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010105&docId=430453511
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010106&docId=430454179
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010107&docId=430406751
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010104&docId=430406275
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010105&docId=430406219
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70101&docId=430405474
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010104&docId=430403650
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010101&docId=430402773
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010102&docId=430394144
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010102&docId=430393575
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010107&docId=430393236
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010105&docId=430390824
h

https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7011403&docId=430273188
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7011401&docId=430177069
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7011403&docId=430384117
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7011403&docId=430382260
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7011404&docId=430442973
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7011403&docId=430446336
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7011404&docId=430414089
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7011401&docId=430446501
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7011403&docId=430447437
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7011404&docId=430388173
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7011404&docId=430368312
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7011404&docId=430388663
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70117&docId=430487369
h

https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70102&docId=430523290
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70102&docId=430535340
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70102&docId=430536591
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70103&docId=430538659
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70102&docId=430546075
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70103&docId=430589103
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70102&docId=430547560
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70102&docId=430551713
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70102&docId=430552848
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70102&docId=430554303
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70102&docId=430561003
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70102&docId=430564628
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70102&docId=430563352
https://kin.naver.com//qn

https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70120&docId=430578943
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010101&docId=430580492
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010101&docId=430580361
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70115&docId=430579976
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010102&docId=430578738
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010104&docId=430577999
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70115&docId=430574558
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010108&docId=430569916
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70112&docId=430557832
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70112&docId=430558217
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70115&docId=430574277
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70115&docId=430573884
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430569182
https://kin.nav

https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70109&docId=430547239
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010104&docId=430555985
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430542628
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=70110&docId=430550951
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010104&docId=430536783
https://kin.naver.com//qna/detail.naver?d1id=7&dirId=7010104&docId=430539722


In [109]:
file_dir = '/naver_jisik/'
file_name = 'qus_ans_'+ cal_std_day(0)+'.json'
res = {
    'meta':{
        'desc':'지식인 주관적 텍스트 추출',
        'cols':{
             'qus_title':'질문제목','qus_dir':'카테고리','qus_doc_id':'의사_고유번호','qus_doc':'의사이름',
            'qus_content':'질문내용','qus_answer':'답변'
        },
        'std_day':cal_std_day(0)
    },
    'data':data
}
res
client.write(file_dir+file_name,json.dumps(res,ensure_ascii=False),encoding='utf-8')

{'meta': {'desc': '지식인 주관적 텍스트 추출',
  'cols': {'qus_title': '질문제목',
   'qus_dir': '카테고리',
   'qus_doc_id': '의사_고유번호',
   'qus_doc': '의사이름',
   'qus_content': '질문내용',
   'qus_answer': '답변'},
  'std_day': '2022-10-16'},
 'data': [{'qus_title': '쌍수풀기에 관하여',
   'qus_dir': '성형외과',
   'qus_doc_id': 'T5Xj8cS9%2F3z5%2FyX49v%2BsGD7EfuEsCcGuym3fGUJ0jh0%3D',
   'qus_doc': '현경배',
   'qus_content': '안검하수쌍수를 해서 시간이 흘렀습니다과도하게 병행되어 첨에 점막이 뒤집어보일 정도였지만이제는 조금내려왔습니다하지만 점막이 여전히보이는상태입니다..처음에아웃라인 같이 된쌍수가 서서히 좀풀리면서그위에 큰쌍커풀이 생기면서 보기싫은 두컵풀이 생겨나기 시작했습니다! 그리고비대칭으로 눈을 잡아서 그런가풀릴때도 덜묵은 쪽이 더풀리더라구요 . .지방을 너무많이 제거 했는진 몰라도 눈커풀이 소시지 쌍커풀처럼 너무 두툼하고 제눈이 제눈같지 않고 무겁고 불편했습니다TT쌍수하기전사진처럼 돌아갈수있을까요?아님 속쌍으로 낮출 수 있나요?보통 낮추는 수술을 권하시겠지만 왠지 낮춰도 달라지지 않을 것같아요..저의 몰골은 참혹하다고 생각되거든요집도하신의사가 너무 심하게 망쳐나서다른의사님들이 신이아닌이상 저의눈을 돌릴 수 있을지 모르겠습니다..☆첫번째사진은 눈을 밑으로 내리깔았을때이고두번째는 쌍수하기전이고세번째는 눈을 치켜떴을 때입니다',
   'qus_answer': '안녕하세요. 대한의사협회·네이버 지식iN 상담의사 현경배 입니다.  올려주신 내용과 사진 잘 검토해 보았습니다.  심한 소세지 쌍꺼풀로 보이고, 이런 경우 쌍꺼풀 푸는 수술을 잘하면 자연스럽게 교정이 가능할 것 같습니다.   너무 절망하지 

In [118]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row

In [113]:
file_name = '/naver_jisik/' + 'qus_ans_'+ cal_std_day(0)+'.json'
jisik_json = spark.read.json(file_name)

In [119]:
data = []
for r1 in jisik_json.select(jisik_json.data, jisik_json.meta.std_day).toLocalIterator():
    for r2 in r1.data:
        temp = r2.asDict()
        temp['std_day'] = r1['meta.std_day'] 
        data.append(Row(**temp))
df_jisik = spark.createDataFrame(data)
df_jisik

DataFrame[qus_answer: string, qus_content: string, qus_dir: string, qus_doc: string, qus_doc_id: string, qus_title: string, std_day: string]

In [130]:
df_jisik.count()

768

In [123]:
df_jisik.select(df_jisik.qus_title, df_jisik.qus_content, df_jisik.qus_answer).show()

+----------------------------------+-----------------------------------+----------------------------------+
|                         qus_title|                        qus_content|                        qus_answer|
+----------------------------------+-----------------------------------+----------------------------------+
|                 쌍수풀기에 관하여| 안검하수쌍수를 해서 시간이 흘렀...|안녕하세요. 대한의사협회·네이버...|
|                     팔자주름 필러|  팔자주름 때문에 두달전에 팔자 ...|  안녕하세요 네이버 지식IN,대한...|
|                       눈꺼풀 처짐| 안녕하세요제가 눈꺼풀이 처져 있...|안녕하세요. 대한성형외과의사회·...|
|                   안면비대칭 치료|                         No Content|  안녕하세요 네이버 지식IN,대한...|
|   제로큐탄 , 항생제 같이 복용 ...|  치과에서 사랑니 뽑고 이렇게 약...| 안녕하세요. 하이닥-네이버 지식...|
| 광대주사(윤곽주사) 부작용 알려...| 제가 광대주사를 맞으려고 하는데...| 안녕하세요. 하이닥-네이버 지식...|
|               실리콘 제거 후 모기|    실리콘 제거한 지 25일 째인데...| 안녕하세요. 하이닥-네이버 지식...|
|           원장님 재수술가능할까요| 정말 절망적이여서요 눈위가 저렇...| 안녕하세요. 하이닥-네이버 지식...|
|            절개쌍수 흉이 심할경우| 줄기세포 치료 받으면 나아질까요...| 안녕하세요. 하이닥-네이버 지식...|
|      

In [133]:
# 맞춤법 교정 

pd_jisik = df_jisik.toPandas()
pd_jisik

,qus_answer,qus_content,qus_dir,qus_doc,qus_doc_id,qus_title,std_day
0,안녕하세요. 대한의사협회·네이버 지식iN 상담의사 현경배 입니다. 올려주신 내용과...,안검하수쌍수를 해서 시간이 흘렀습니다과도하게 병행되어 첨에 점막이 뒤집어보일 정도였...,성형외과,현경배,T5Xj8cS9%2F3z5%2FyX49v%2BsGD7EfuEsCcGuym3fGUJ0...,쌍수풀기에 관하여,2022-10-16
1,"안녕하세요 네이버 지식IN,대한의사 협회 공식 성형외과 답변의, 서울 압구정역 ""오...",팔자주름 때문에 두달전에 팔자 지방이식 그리고 2 주전에 스컬트라까지했는데 오히려 ...,성형외과,김영조,ivN%2B5Asi7CP717cEauHD78CX79cbV0HYeF3J%2FMPe7s...,팔자주름 필러,2022-10-16
2,안녕하세요. 대한성형외과의사회·네이버 지식iN 상담의사 박유길 입니다. 윗 눈꺼...,"안녕하세요제가 눈꺼풀이 처져 있는데요,이마의 힘으로 눈을 뜨지는 않은데 왜 그런걸까...",성형외과,박유길,F6YyWEBi4A48%2B6KlN8DpgbxX80dsYAWKbBfaE1fTNes%3D,눈꺼풀 처짐,2022-10-16
3,"안녕하세요 네이버 지식IN,대한의사 협회 성형외과 답변의, 서울 압구정역 ""오렌지 ...",No Content,성형외과,김영조,ivN%2B5Asi7CP717cEauHD78CX79cbV0HYeF3J%2FMPe7s...,안면비대칭 치료,2022-10-16
4,안녕하세요. 하이닥-네이버 지식iN 상담의 이상준 입니다. 해당 항생제와는 상호작...,치과에서 사랑니 뽑고 이렇게 약 처방 받았는데요!기존에 제로큐탄 하루에 2알씩 복용...,피부과,이상준,eWay0Vcfd7JAVzqKYn3t2oAUygpGzFSUyp2Dw9JGwjc%3D,"제로큐탄 , 항생제 같이 복용 하도 되나요?",2022-10-16
...,...,...,...,...,...,...,...
763,안녕하세요. 대한의사협회·네이버 지식iN 상담의사 김철수 입니다. 5년 이상 기간이...,CT나 MRI나 PET-CT 상 나타나지 않던 암세포가 떠다녀서 장기나 뇌 쪽으로 ...,혈액종양내과,김철수,ZpLHwcZs6VN8eRB64%2FqBWJ45B1HSM8srkcTGsmusWwA%3D,근데 의학적으로 완전 관해 후 5년 완치라는 부분이 모순인게 5년...,2022-10-16
764,안녕하세요. 대한성형외과의사회·네이버 지식iN 상담의사 박유길 입니다. 설령 실...,쌍수한지는 2년 10개월 됐는데 갑자기 고개 돌리면서 눈을 세게 감은건 아닌데 눈을...,성형외과,박유길,F6YyWEBi4A48%2B6KlN8DpgbxX80dsYAWKbBfaE1fTNes%3D,쌍수하고 소리,2022-10-16
765,안녕하세요. 대한의사협회·네이버 지식iN 상담의사 박상은 입니다. ◆ 질문하신 내...,광대밑? 에 파인곳에 미세지방이식했는데(2일차)약간 까만곳이 의사가 싸인펜으로 표시...,성형외과,박상은,AVD2hYA4aOzjgv9LwdXHmBZ4pD9TCxhMkUedT4UhvGg%3D,미세지방이식 질문,2022-10-16
766,안녕하세요. 대한의사협회·네이버 지식iN 상담의사 김철수 입니다. 실비보험인가요? ...,"안녕하세요.경계성종양으로 부신종양절제 수술받고 D44.7진단받았습니다.""주계약 경계...",혈액종양내과,김철수,ZpLHwcZs6VN8eRB64%2FqBWJ45B1HSM8srkcTGsmusWwA%3D,부신경절의 미상신생물D44.7 경계성종양수술비,2022-10-16


In [132]:
# 내과
internal_medicine = df_jisik.where(df_jisik.qus_answer.contains('내과'))
# 정형외과
orthopedics = df_jisik.where(df_jisik.qus_answer.contains('정형외과'))
# 피부과
dermatology = df_jisik.where(df_jisik.qus_answer.contains('피부과'))
# 산부인과
ob_gyne = df_jisik.where(df_jisik.qus_answer.contains('산부인과'))
# 신경외과
neurosurgery = df_jisik.where(df_jisik.qus_answer.contains('신경외과'))
# 안과
Ophthalmology = df_jisik.where(df_jisik.qus_answer.contains('안과'))

+----------------------------------+---------------------------------+------------+-------+--------------------+------------------------------------+----------+
|                        qus_answer|                      qus_content|     qus_dir|qus_doc|          qus_doc_id|                           qus_title|   std_day|
+----------------------------------+---------------------------------+------------+-------+--------------------+------------------------------------+----------+
|  증상이 1주일 이상 지속되는 경...| 일주일 넘게 배가 너무 아프고 ...|  가정의학과| 서민석|0eTroy4NXt%2F3Pf9...|      배가 계속 아프고 속이 안좋아요|2022-10-16|
| 안녕하세요. 위염의 가능성이 있...|제가 복부 명치 아래쪽에 두근거...|        내과| 조영욱|8Y802qnN3i9vBpzDj...|     제가 복부쪽 두근거림이 심합니다|2022-10-16|
| 안녕하세요. 과로와 스트레스 불...|체기가 심한데 약이나 탄산수를 ...|  소화기내과| 조영욱|8Y802qnN3i9vBpzDj...|           체기가 심하고 낫질 않아요|2022-10-16|
| 안녕하세요. 위염의 가능성이 있...|제가 복부 명치 아래쪽에 두근거...|        내과| 조영욱|8Y802qnN3i9vBpzDj...|   제가 복부 명치 아래쪽에 두근거...|2022-10-16|
|안녕하세요. 소화기내과 전문의입...|제 친구가 어제 빈속으로 닭갈비...|  소화기내과| 